In [6]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import h5py
import os
import seaborn as sns


In [14]:
from scf_runs import run_casci, run_lda_he
scf_checkfile, opt_checkfile, mf_lda = run_lda_he()
ci_checkfile, mc = run_casci(scf_checkfile, nroots=1, ncas =3, nelecas = (1,1))

He atom neutral LDA spin=0
HF
converged SCF energy = -2.86151422722833  <S^2> = 2.4424907e-15  2S+1 = 1
LDA
converged SCF energy = -2.83456474290722  <S^2> = 3.1086245e-15  2S+1 = 1
CASCI nelecas up/down (1, 1)
CASCI E = -2.86838486726576  E(CI) = -2.86838486726576  S^2 = 0.0000000
Available output from CASCI: <KeysViewHDF5 ['ci', 'fci', 'mo_coeff', 'ncas', 'nelecas']>


In [15]:
import bosonrecipes
import importlib, bosonrecipes, linemin, bosonwftools, mc, bosonslater, bosonaccumulators, stochastic_reconfiguration
modules = [importlib, bosonrecipes, linemin, bosonwftools, mc, bosonslater, bosonaccumulators, stochastic_reconfiguration]
for m in modules:
    importlib.reload(m)


In [16]:
# abvmcopt_file = 'abvmc-j-opt-he.hdf5'
# for fname in [abvmcopt_file]:
#     if os.path.isfile(fname):
#         os.remove(fname)
# num_it = 12
# bosonrecipes.ABOPTIMIZE(scf_checkfile, 
#                         abvmcopt_file, 
#                         ci_checkfile   = ci_checkfile,
#                         max_iterations = num_it, 
#                         jastrow_kws    = {"ion_cusp":False, 'na':0},
#                         verbose        = True,                            
#                         nconfig        = 1000)  


In [20]:
import concurrent
abvmcopt_file = 'abvmc-j-opt-he.hdf5'
num_it = 20
ncore = 16 
for fname in [abvmcopt_file]:
    if os.path.isfile(fname):
        os.remove(fname)
with concurrent.futures.ProcessPoolExecutor(max_workers=ncore) as client:
    bosonrecipes.ABOPTIMIZE(scf_checkfile, 
                    abvmcopt_file, 
                    ci_checkfile = ci_checkfile,
                    max_iterations=num_it, 
                    verbose=True,  
                    load_parameters = None, 
                    jastrow_kws={"ion_cusp":False, 'na':0},
                    client = client, 
                    npartitions=ncore,                            
                    nconfig=1000)


Using spherical guess
starting warmup
finished warmup
----------vmc done
Gradient norm:  0.2048815130331463
Dot product between gradient and SR step:  0.9443847971984491
[ 0.06344222  0.0631904   0.10281001  0.10109849  0.08080571  0.07897104
  0.03425822  0.03341312  0.         -0.02798595  0.          0.
 -0.0789084   0.          0.         -0.0749451   0.        ]
descent en -2.63371183601926 0.002293851035138984
energies from correlated sampling [-2.5583279  -2.58993966 -2.62040068 -2.64968705 -2.67776756]
----------vmc done
Gradient norm:  0.1825710691395628
Dot product between gradient and SR step:  0.9306081105840373
[ 0.12296024  0.12178771  0.19540156  0.19225341  0.14355224  0.14121111
  0.05082962  0.05045236  0.         -0.03997139  0.          0.
 -0.12818022  0.          0.         -0.12064943  0.        ]
descent en -2.6978114955908845 0.0018856842792716693
energies from correlated sampling [-2.6494976  -2.66807989 -2.68557357 -2.70191706 -2.71704836]
----------vmc done


KeyboardInterrupt: 

In [ ]:
import pyqmc.api as pyq
df = pyq.read_opt(abvmcopt_file)
df

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df['energy'], '-ok')